# Strategy

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name bonus')


class Item:

    def __init__(self, product, quantity, price) -> None:
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:

    def __init__(self, customer, cart, promotion=None) -> None:
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def after_prom(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self) -> str:
        return f'<Order total: {self.total():.2f} after: {self.after_prom():.2f}>'


class Promoution(ABC):

    @abstractmethod
    def discount(self, order):
        """Generate discount"""


class Bonus_promo(Promoution):

    def discount(self, order):
        return order.total() * 0.05 if order.customer.bonus >= 1000 else 0


class Value_promo(Promoution):

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20: discount *= item.total() * 0.1
        return discount


class Large_promo(Promoution):

    def discount(self, order):
        unique_items = {item.product for item in order.cart}
        if len(unique_items) >= 5: return order.total() * 0.07
        else: return 0

In [2]:
ann = Customer(name='Ann', bonus=0)
joe = Customer(name='Joe', bonus=2000)
cart1 = [
    Item('apple', 10, 5),
    Item('banana', 5, 8),
    Item('toy', 1, 50),
    Item('car', 2, 100),
    Item('banana', 9, 8),
]
cart2 = [
    Item('apple', 9, 5),
    Item('banana', 5, 8),
    Item('toy', 1, 50),
    Item('car', 2, 100),
    Item('banana', 19, 8),
]
Order(joe,cart1,Bonus_promo())

<Order total: 412.00 after: 391.40>

In [3]:
class Order:

    def __init__(self, customer, cart, promotion=None) -> None:
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def after_prom(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self) -> str:
        return f'<Order total: {self.total():.2f} after: {self.after_prom():.2f}>'

def Bonus_promo(order):
    return order.total() * 0.05 if order.customer.bonus >= 1000 else 0

def Value_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20: discount *= item.total() * 0.1
    return discount

def Large_promo(order):
    unique_items = {item.product for item in order.cart}
    if len(unique_items) >= 5: return order.total() * 0.07
    else: return 0
    
def Best_promo(order):
    return max(promo(order)for promo in (Bonus_promo,Value_promo,Large_promo))


In [4]:
Order(joe,cart2,Value_promo)

<Order total: 487.00 after: 487.00>

In [5]:
Order(joe,cart2,Best_promo)

<Order total: 487.00 after: 462.65>

# Command

In [13]:
class Commands:

    @staticmethod
    def print_com():
        print('Print command')

    @staticmethod
    def printValu_com():
        value = 5
        print(f'Value: {value:.2f}')

    @staticmethod
    def return_com():
        return 'return sting'


class MacroCommand:

    def __init__(self, commads):
        self.commads = list(commads)

    def __call__(self):
        return_list=[]
        for com in self.commads:
            return_list.append(com())
        return return_list

In [14]:
macro1 = MacroCommand(commads=[
    Commands.print_com,
    Commands.printValu_com,
    Commands.return_com,
], )


In [15]:
a= macro1()
print('*'*20,f'\n {a}')

Print command
Value: 5.00
******************** 
 [None, None, 'return sting']
